In [323]:
import pandas as pd
from collections import defaultdict

from bokeh.io import output_notebook, show, save
from bokeh.plotting import figure
from bokeh.layouts import column, gridplot
from bokeh.models import HoverTool, Spacer

output_notebook()

Loading BokehJS ...

In [202]:
points = pd.read_csv("../data/points.csv")
events = pd.read_csv("../data/season_events.csv")

points.loc[points["column"] == "merge", "value"] = 5
points.loc[points["column"] == "idol", "value"] = 5
points.loc[points["column"] == "immunity", "value"] = 5
points.loc[points["column"] == "winner", "value"] = 20


realized_points = {}
potential_points = {}

for person in points.person.unique():
    realized_points[person] = [0]

for person in points.person.unique():
    potential_points[person] = [0]


In [203]:

# 13 episodes total (1-based)
for episode in range(1, 14):
    episode_events = events.query("episode == @episode")
    
    # merge
    if episode == 7:
        merge_pts = points.query("column == 'merge'").groupby("person")["value"].sum()
        points = points.loc[points["column"] != "merge", :]
    else:
        merge_pts = pd.Series()
    
    # individual immunity
    imm_events = events.query("(event == 'immunity') & (episode == @episode)")
    imm_points = points.loc[[player in imm_events.player for player in points.player], :].groupby("person")["value"].sum()
    points = points.loc[~(pd.Series([player in imm_events.player for player in points.player]) & (points["column"] == "immunity")), :]

    
    # idol
    idol_events = events.query("(event == 'idol') & (episode == @episode)")
    idol_points = points.loc[[player in idol_events.player for player in points.player], :].groupby("person")["value"].sum()
    points = points.loc[~(pd.Series([player in idol_events.player for player in points.player]) & (points["column"] == "idol")), :]

    # final tribal
    f_tribal_events = events.query("(event == 'final_tribal') & (episode == @episode)")
    final_tribal_points = points.loc[[player in f_tribal_events.player for player in points.player], :].groupby("person")["value"].sum()
    points = points.loc[~(pd.Series([player in f_tribal_events.player for player in points.player]) & (points["column"] == "final_tribal")), :]

    # elimination
    eliminated = episode_events.query("event == 'eliminated'").player.to_list()
    for player in eliminated:
        points = points.query("player not in @player")

    # realized_points

    new_points = pd.concat([final_tribal_points, idol_points, merge_pts, imm_points], axis=1).sum(axis=1)

    for person in realized_points.keys():
        if person in new_points.index:
            realized_points[person].append(realized_points[person][-1] + new_points[person])
        else:
            realized_points[person].append(realized_points[person][-1])

    # potential points
    ppoints = points.groupby("person")["value"].sum()
    for person in potential_points.keys():
        if person in ppoints.index:
            potential_points[person].append(realized_points[person][-1] + ppoints[person])
        else:
            potential_points[person].append(realized_points[person][-1])



In [204]:
realized_points = pd.DataFrame(realized_points)
realized_points

,Nava,Giulia,Quinn,Jen,Anthony,Ryan
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0
7,45.0,40.0,35.0,40.0,35.0,40.0
8,45.0,40.0,35.0,40.0,35.0,40.0
9,45.0,40.0,35.0,40.0,35.0,40.0


In [205]:
potential_points = pd.DataFrame(potential_points)
potential_points

,Nava,Giulia,Quinn,Jen,Anthony,Ryan
0,0.0,0.0,0.0,0.0,0.0,0.0
1,170.0,170.0,145.0,170.0,170.0,170.0
2,170.0,170.0,140.0,165.0,150.0,170.0
3,160.0,155.0,120.0,165.0,140.0,153.0
4,160.0,150.0,120.0,155.0,135.0,148.0
5,160.0,150.0,120.0,140.0,135.0,148.0
6,160.0,150.0,115.0,140.0,135.0,143.0
7,160.0,130.0,60.0,110.0,125.0,139.0
8,160.0,110.0,60.0,85.0,125.0,139.0
9,155.0,80.0,45.0,75.0,115.0,124.0


In [341]:
current_episode = 10




colors = ["red", "darkorange", "green", "blue", "indigo", "purple"]

plot_list = []


p = figure(height=600, width=400, 
              x_range=(1,10), y_range=(0, potential_points.max().max() + 20), tools=["save"],
              x_axis_label = "Episode", y_axis_label="Points", margin=(0,50,0,0))



for idx, person in enumerate(realized_points.columns):
    c = p.varea(x=list(range(0,14)), y1=realized_points.loc[:, person], y2=potential_points.loc[:, person], 
           legend_label=person, fill_color=colors[idx], alpha=0.1, muted_alpha=0)
    a = p.line(x=list(range(0,14)), y=realized_points.loc[:, person], line_width=6,
           legend_label=person, line_color=colors[idx], alpha=1, muted_alpha=0)
    b = p.line(x=list(range(0,14)), y=potential_points.loc[:, person], line_width=6,
           legend_label=person, line_color=colors[idx], alpha=0.5, line_dash="dashed", muted_alpha=0)
    
    plot_list.append(p)

    if idx > 0:
       a.muted = True
       b.muted = True
       c.muted = True

p.legend.location = "left"
p.title = "Click the legend to show data\nSolid line shows current points\nDotted line shows potential points"
p.title.text_font_size = "30pt"

p.legend.click_policy = "mute"
p.xaxis.ticker.desired_num_ticks = 13
p.xaxis.minor_tick_line_alpha = 0
p.xaxis.major_label_text_font_size = "30pt"
p.xaxis.axis_label_text_font_size = "35pt"

p.yaxis.ticker.desired_num_ticks = 10
p.yaxis.major_label_text_font_size = "30pt"
p.yaxis.axis_label_text_font_size = "35pt"

p.legend.label_text_font_size = "35pt"


layout = gridplot([[p]], sizing_mode='stretch_both')
show(layout)
save(layout, "../index.html")



/var/folders/3t/77tvz6w530g5vg1vrt94l3hc0000gn/T/ipykernel_15507/1709705951.py:51: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  save(layout, "../index.html")
/var/folders/3t/77tvz6w530g5vg1vrt94l3hc0000gn/T/ipykernel_15507/1709705951.py:51: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  save(layout, "../index.html")


'/Users/anthony/Desktop/survivor/index.html'